In [20]:
import os
import re
import json
import pandas as pd
from pathlib import Path

In [32]:
PROJECT_DIR_PATH = Path.cwd()
DATA_DIR_PATH = PROJECT_DIR_PATH / "data"
INTERNAL_DATA_DIR_PATH = DATA_DIR_PATH / "internal"
PANEL_DATA_DIR_PATH = INTERNAL_DATA_DIR_PATH / "2. SRI시민패널조사"

print(PROJECT_DIR_PATH)
print(DATA_DIR_PATH)
print(INTERNAL_DATA_DIR_PATH)
print(PANEL_DATA_DIR_PATH)

d:\workspace\Bus_suwon
d:\workspace\Bus_suwon\data
d:\workspace\Bus_suwon\data\internal
d:\workspace\Bus_suwon\data\internal\2. SRI시민패널조사


In [33]:
# 1) 경로 정의 (앞에 슬래시 금지!)
files = [
    ("2024년 3분기 패널조사", "2024년 3분기 패널조사 데이터_코드북.xlsx"),
    ("2024년 4분기 패널조사", "2024년 4분기 패널조사 데이터_코드북.xlsx"),
    ("2025년 1분기 패널조사", "2025년 1분기 패널조사_코드북.xlsx"),
    ("2025년 2분기 패널조사", "2025년 2분기 패널조사_코드북.xlsx"),
]

file_paths = [PANEL_DATA_DIR_PATH / subdir / fname for subdir, fname in files]

# 2) 존재 확인
for p in file_paths:
    print(p, "->", p.exists())

# 3) JSON 출력 디렉터리
out_dir = PROJECT_DIR_PATH / "output" / "codebooks"
out_dir.mkdir(parents=True, exist_ok=True)


d:\workspace\Bus_suwon\data\internal\2. SRI시민패널조사\2024년 3분기 패널조사\2024년 3분기 패널조사 데이터_코드북.xlsx -> True
d:\workspace\Bus_suwon\data\internal\2. SRI시민패널조사\2024년 4분기 패널조사\2024년 4분기 패널조사 데이터_코드북.xlsx -> True
d:\workspace\Bus_suwon\data\internal\2. SRI시민패널조사\2025년 1분기 패널조사\2025년 1분기 패널조사_코드북.xlsx -> True
d:\workspace\Bus_suwon\data\internal\2. SRI시민패널조사\2025년 2분기 패널조사\2025년 2분기 패널조사_코드북.xlsx -> True


In [41]:
# 원형 숫자 → 숫자
circled_map = str.maketrans({
    '①':'1','②':'2','③':'3','④':'4','⑤':'5','⑥':'6','⑦':'7','⑧':'8','⑨':'9','⑩':'10',
    '⑪':'11','⑫':'12','⑬':'13','⑭':'14','⑮':'15','⑯':'16','⑰':'17','⑱':'18','⑲':'19','⑳':'20'
})
def norm_text(s: str) -> str:
    if s is None:
        return ""
    s = str(s)
    if s.lower() == "nan":
        return ""
    s = s.replace("\r\n", "\n").replace("\r", "\n").translate(circled_map)
    s = re.sub(r"[ \t]+", " ", s)
    return s.strip()

In [42]:
# "1 내용", "1) 내용", "1. 내용", "1 - 내용" 등
CHOICE_LINE = re.compile(r"^\s*(\d{1,3})[)\.\-]?\s*(.+?)\s*$")
def parse_choices(text):
    text = norm_text(text)
    if not text: return []
    out = []
    for line in text.split("\n"):
        m = CHOICE_LINE.match(line.strip())
        if m:
            out.append({"code": int(m.group(1)), "label": m.group(2).strip()})
    return out

In [43]:
def detect_type(var, question, choices, unit_text):
    if var.endswith("_op") or "오픈" in (question or ""): return "open"
    if choices: return "single"
    if unit_text: return "numeric"
    return "text"

In [44]:
def build_codebook_json(xlsx: Path):
    df = pd.read_excel(xlsx, dtype=str)
    df.columns = [c.strip() for c in df.columns]
    # 컬럼 해석
    def pick(*cands): 
        for c in cands:
            if c in df.columns: return c
        return None
    c_sec = pick("구분","대분류","섹션")
    c_var = pick("변수명","변수","Var","var")
    c_q   = pick("질문","문항","Question")
    c_val = pick("변수값/단위","값/단위","값","코드값","응답값","라벨")
    assert all([c_sec,c_var,c_q,c_val]), f"필수 컬럼 누락: {xlsx}, columns={df.columns.tolist()}"

    df[c_sec] = df[c_sec].fillna(method="ffill")
    df[c_var] = df[c_var].astype(str).str.strip()
    df[c_q]   = df[c_q].apply(norm_text)
    df[c_val] = df[c_val].apply(norm_text)

    data = {
        "meta": {"source_file": xlsx.name, "quarter": xlsx.stem},
        "sections": {},
        "variables": {}
    }
    for _, r in df.iterrows():
        var = norm_text(r[c_var])
        if not var or var.lower()=="nan": continue
        sec = norm_text(r[c_sec])
        q   = norm_text(r[c_q])
        val = norm_text(r[c_val])
        choices = parse_choices(val)
        unit_text = "" if choices else val
        vtype = detect_type(var, q, choices, unit_text)

        ent = {"section":sec,"name":var,"label":q,"type":vtype,"unit":(unit_text or None),"choices":(choices or None)}
        data["variables"][var] = ent
        data["sections"].setdefault(sec, []).append(var)

    data["survey"] = [{"section": s, "items":[data["variables"][v] for v in vs]} for s,vs in data["sections"].items()]
    return data

In [46]:
# 4) 변환 실행
created = []
for p in file_paths:
    if not p.exists():
        print(f"[WARN] Not found: {p}")
        continue
    obj = build_codebook_json(p)
    out_path = out_dir / (p.stem + ".json")
    out_path.write_text(json.dumps(obj, ensure_ascii=False, indent=2), encoding="utf-8")
    created.append(out_path)

C:\Users\wsm\AppData\Local\Temp\ipykernel_12948\153224941.py:15: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[c_sec] = df[c_sec].fillna(method="ffill")
C:\Users\wsm\AppData\Local\Temp\ipykernel_12948\153224941.py:15: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[c_sec] = df[c_sec].fillna(method="ffill")
C:\Users\wsm\AppData\Local\Temp\ipykernel_12948\153224941.py:15: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[c_sec] = df[c_sec].fillna(method="ffill")
C:\Users\wsm\AppData\Local\Temp\ipykernel_12948\153224941.py:15: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[c_sec] = df[c_sec].fillna(method="ffill")


In [47]:
# 5) 통합본
combined = {"meta":{"files":[x.name for x in created]}, "quarters": {}}
for p in created:
    combined["quarters"][p.stem] = json.loads(p.read_text(encoding="utf-8"))

combined_path = out_dir / "ALL_quarters_codebook.json"
combined_path.write_text(json.dumps(combined, ensure_ascii=False, indent=2), encoding="utf-8")

print("created:", [str(x) for x in created])
print("combined:", str(combined_path))

created: ['d:\\workspace\\Bus_suwon\\output\\codebooks\\2024년 3분기 패널조사 데이터_코드북.json', 'd:\\workspace\\Bus_suwon\\output\\codebooks\\2024년 4분기 패널조사 데이터_코드북.json', 'd:\\workspace\\Bus_suwon\\output\\codebooks\\2025년 1분기 패널조사_코드북.json', 'd:\\workspace\\Bus_suwon\\output\\codebooks\\2025년 2분기 패널조사_코드북.json']
combined: d:\workspace\Bus_suwon\output\codebooks\ALL_quarters_codebook.json
